In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import re
import datetime
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import norm, probplot
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from arch.unitroot import VarianceRatio
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller 
from scipy.stats import genpareto
from sklearn.utils import resample
from scipy.optimize import minimize
from scipy.special import expit, logit
from sklearn.linear_model import LogisticRegression
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from scipy.optimize import minimize
from scipy.stats import genpareto
from scipy.optimize import differential_evolution
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from scipy.optimize import differential_evolution
from scipy.stats import gumbel_r
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import RFE
import ta
import shap
from xgboost.callback import EarlyStopping
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from scipy.stats import entropy

import xgboost as xgb
from sklearn.preprocessing import LabelBinarizer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
from tensorflow.keras.models import Sequential

ModuleNotFoundError: No module named 'tensorflow.keras'

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


2024-08-07 17:54:00.264333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. This error is frequently encountered on macOS when running an x86 Python installation on ARM hardware. In this case, try installing an ARM build of Python. Otherwise, you may be able work around this issue by building jaxlib from source.

In [1]:
!pip uninstall -y tensorflow

Found existing installation: tensorflow 2.13.1
Uninstalling tensorflow-2.13.1:
  Successfully uninstalled tensorflow-2.13.1


In [2]:
!pip install tensorflow

  Using cached tensorflow-2.13.1-cp38-cp38-macosx_10_15_x86_64.whl.metadata (3.2 kB)
Using cached tensorflow-2.13.1-cp38-cp38-macosx_10_15_x86_64.whl (216.2 MB)


In [1]:
import tensorflow as tf

: 

: 

: 

: 

: 

: 

: 

In [1]:
!pip install --upgrade numpy wheel setuptools

  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.


In [4]:
!pip install --upgrade wheel setuptools
!pip install --upgrade keras_preprocessing

In [3]:
!pip uninstall -y numpy
!pip install numpy==1.24.3
!pip install tensorflow==2.13.1 --upgrade



Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.3-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)


In [1]:

import numpy as np

print("NumPy version:", np.__version__)


NumPy version: 1.24.3


In [4]:
!pip install tensorflow==2.13.1 --upgrade


In [ ]:
from tensorflow.keras.layers import LSTM, Dense

: 

#### Forecasting directional movement of Crypto and stock data using LSTM with technical and stochastic volatility indicators

- In comparison with stock market, crypto is a special financial market that entails both high risks and high profit opportunities. Like stock, trading in cryptocurrencies often involves predicting the direction of the price movement (up or down). Traders can profit from buying low and selling high or short-selling high and buying back lower.

- In this work, we used a popular deep learning tool called “long short-term memory” (LSTM), which has been shown to be very effective in many time-series forecasting problems, to make direction predictions in crypto and stock markets. 

- In recent years, deep learning tools, such as long short-term memory (LSTM), have become popular and have been found to be effective for many time-series forecasting problems. In general, such problems focus on determining the future values of time- series data with high accuracy. However, in direction prediction problems, accuracy can- not be defined as simply the difference between actual and predicted values. Therefore, a novel rule-based decision layer needs to be added after obtaining predictions from LSTMs.

- Baseline LSTMs: As a baseline, SV-LSTM and TI-LSTM were tested separately. Also, by combining all of the features of these two into a single model, we generated a third baseline model: SV-TI-LSTM.

- Proposed hybrid LSTM model: Our proposed model does not combine the features of the two baseline LSTMs into a single model. Instead, we propose a rule-based decision mechanism that acts as a kind of postprocessing; it is used to combine the results of the baselines into a final decision.

- Model set up: The training phase was carried out with different numbers of iterations (50, 100, and 150); Only when a difference between two consecutive data points is greater/less than the threshold will the next data point be labeled as increase/decrease; 

- Post-processing rule: if any model's prediction is class-noact, the final decision is class-noact; if both models agree on the label, set the final decision as the label; if the predictions of the two models are different, let the final decision the one whose predition has higher prob, if the prob is the same set the prediction of the TI-LSTM model --> this is a conservative approach as it reduces the number of trades and favors only high-accuracy predictions. 

### Import Data

In [ ]:
btc_min_return = pd.read_csv('btc_min_return_selected.csv')
btc_min_return.index =pd.to_datetime(btc_min_return['timestamp'])
btc_min_return.drop(columns=['timestamp'],inplace=True)
btc_min_return['return_scaled'] = btc_min_return['return']*100
print(btc_min_return.columns)

features = btc_min_return.columns.to_list()
features.remove('return_scaled')
TI_features = features[:6]+features[-17:]
SV_features = features[:-17]

### Generate labels

In [ ]:
# Create the target variable -- Quantile based threshold
upper_quantile = btc_min_return['return_scaled'].quantile(0.7)
upper_quantile = btc_min_return['return_scaled'].quantile(0.3)
btc_min_return['target_quantile'] = 0
btc_min_return.loc[btc_min_return['return_scaled'].shift(-1) > upper_quantile, 'target_quantile'] = 1
btc_min_return.loc[btc_min_return['return_scaled'].shift(-1) < lower_quantile, 'target_quantile'] = -1
btc_min_return = btc_min_return.dropna()
print([upper_quantile,lower_quantile])

In [ ]:
print(len(btc_min_return)/200/60/24) # roughly ten days per bin
number_of_bins = 200
bin_counts, bin_edges = np.histogram(btc_min_return['return_scaled'], bins=number_of_bins)

sorted_indices = np.argsort(bin_counts)[::-1]
sorted_bin_counts = bin_counts[sorted_indices]
sorted_bin_edges = bin_edges[sorted_indices]

original_bin_edges = bin_edges
cumulative_counts = np.cumsum(sorted_bin_counts)
total_count = len(btc_min_return)  
threshold_count_upper = 0.7 * total_count
threshold_count_lower = 0.3 * total_count

threshold_bin_index_upper = np.where(cumulative_counts >= threshold_count_upper)[0][0]
threshold_upper_bound = original_bin_edges[sorted_indices[threshold_bin_index_upper] + 1] if threshold_bin_index_upper < len(original_bin_edges) - 1 else original_bin_edges[-1]
threshold_bin_index_lower = np.where(cumulative_counts >= threshold_count_lower)[0][0]
threshold_lower_bound = original_bin_edges[sorted_indices[threshold_bin_index_lower]]
print(f'Upper bound of the threshold value: {threshold_upper_bound}')
print(f'Lower bound of the threshold value: {threshold_lower_bound}')

btc_min_return['target_hist'] = 0
btc_min_return.loc[btc_min_return['return_scaled'].shift(-1) > threshold_upper_bound, 'target_hist'] = 1
btc_min_return.loc[btc_min_return['return_scaled'].shift(-1) < threshold_lower_bound, 'target_hist'] = -1
btc_min_return = btc_min_return.dropna()

# Plotting
plt.figure(figsize=(14, 4))

plt.subplot(1, 2, 1)
plt.hist(btc_min_return['return_scaled'], bins=number_of_bins, edgecolor='k', alpha=0.7)
plt.title('Histogram of BTC Min Returns')
plt.xlabel('Return Scaled')
plt.ylabel('Count')

In [ ]:
# Function to calculate entropy
def calculate_entropy(labels):
    label_counts = np.bincount(labels.astype(int))
    return entropy(label_counts, base=2)

def calculate_best_thresholds(close_diff, threshold_upper_bound):
    best_entropy_upper = -np.inf
    best_threshold_upper = 0

    # Upper threshold search
    temp_threshold = 0
    while temp_threshold < threshold_upper_bound:
        labels = np.zeros(len(close_diff))
        indexes_incr = np.where(close_diff > temp_threshold)[0]
        indexes_decr = np.where(-close_diff > temp_threshold)[0]

        labels[indexes_incr] = 2
        labels[indexes_decr] = 1

        current_entropy = calculate_entropy(labels)

        if current_entropy > best_entropy_upper:
            best_entropy_upper = current_entropy
            best_threshold_upper = temp_threshold

        temp_threshold += 0.00001

    return best_threshold_upper

# Applying the algorithm
Entropy_upper_threshold = calculate_best_thresholds(btc_min_return['return_scaled'], threshold_upper_bound)
print([Entropy_upper_threshold, -Entropy_upper_threshold])
btc_min_return['target_entropy'] = 0
btc_min_return.loc[btc_min_return['return_scaled'].shift(-1) > Entropy_upper_threshold, 'target_entropy'] = 1
btc_min_return.loc[btc_min_return['return_scaled'].shift(-1) < -Entropy_upper_threshold, 'target_entropy'] = -1
btc_min_return = btc_min_return.dropna()

In [ ]:
df = btc_min_return.copy()
X = df[features]
y = df['target_entropy']

# Split the data into training and test sets
cutoff_date = '2022-01-01'
X_train = X[btc_min_return.index < cutoff_date]
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = X[btc_min_return.index >= cutoff_date]
scaler = StandardScaler()
X_test= scaler.fit_transform(X_test)
y_train = y[btc_min_return.index < cutoff_date]
y_test = y[btc_min_return.index >= cutoff_date]


In [ ]:
X_TI = df[TI_features]
X_train_TI = X_TI[df[TI_features].index < cutoff_date]
scaler = StandardScaler()
X_train_TI= scaler.fit_transform(X_train_TI)
X_test_TI = X_TI[df[TI_features].index >= cutoff_date]
scaler = StandardScaler()
X_test_TI= scaler.fit_transform(X_test_TI)

X_SV = df[SV_features]
scaler = StandardScaler()
X_SV= scaler.fit_transform(X_SV)
X_train_SV = X_SV[df[SV_features].index < cutoff_date]
X_test_SV = X_SV[df[SV_features].index >= cutoff_date]

### Evaluation of different features' prediction power

In [ ]:
correlations = X.corrwith(y).abs().sort_values(ascending=False)

plt.figure(figsize=(6, 8))
sns.barplot(x=correlations.values, y=correlations.index, palette='viridis')
plt.title('Feature Correlations with Target')
plt.xlabel('Absolute Correlation')
plt.ylabel('Features')
plt.show()

#### Recursive Feature Elimination (RFE)

- Backward Selection: Train the model using the full set of features then remove the least important feature(s).

In [ ]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=5)
rfe = rfe.fit(X_train, y_train)
ranking = pd.Series(rfe.ranking_, index=X.columns).sort_values()

plt.figure(figsize=(12, 6))
ranking.plot(kind='bar')
plt.xlabel('Features')
plt.ylabel('Ranking')
plt.title('Feature Ranking from Recursive Feature Elimination')
plt.show()

#### Mutual Information 

$I(X,Y) = \sum_{x \in X} \sum_{y \in Y} p(x,y) \log \frac{p(x,y)}{p(x) p(y)}$;

where $p(\cdot)$ denote the probability distribution.

In [ ]:
# Calculate mutual information between each feature and the target
mi = mutual_info_classif(X_train, y_train)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(12, 6))
mi_series.plot(kind='bar')
plt.xlabel('Features')
plt.ylabel('Mutual Information')
plt.title('Mutual Information between Features and Target')
plt.show()

#### XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
xgb_model.fit(X_train, y_encoded)

# Get feature importances
importances = xgb_model.feature_importances_
feature_importances = pd.Series(importances, index=X.columns).sort_values(ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 6))
feature_importances.plot(kind='bar')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances from XGBoost')
plt.show()

### Define Accuracy metrics

- Type I error: FP (reject null while null is true)

- Type II error: FN (not reject null while null is not true)

- Precision: $\frac{TP}{TP + FP}$

- Recall: $\frac{TP}{TP + FN}$

- F1-Score: $2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$

- Accuracy_function:

$\text{Profit}-\text{accuracy} = \frac{TD + TI}{TD + TI + FDN+FIN+FID+FDI}$;

s.t.
TD: the number of true predictions decreases; FDN: the number of predictions of the no-action class decreases; FDI: the number of predictions of the increase class decreases and etc.




In [ ]:
def custom_accuracy(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    true_neg = conf_matrix[0, 0]
    true_pos = conf_matrix[2, 2]
    neutral_pred_as_neg = conf_matrix[1, 0]
    neutral_pred_as_pos = conf_matrix[1, 2]
    neg_pred_as_pos = conf_matrix[0, 2]
    pos_pred_as_neg = conf_matrix[2, 0]
    total = conf_matrix.sum()
    custom_acc = (true_neg + true_pos) / (total - (neutral_pred_as_neg + neutral_pred_as_pos + neg_pred_as_pos + pos_pred_as_neg))
    return custom_acc

### Benchmark Model: Linear Regression Model

In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    conf_matrix = confusion_matrix(y_true, y_pred)
    conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

    plt.figure(figsize=(4, 3))
    sns.heatmap(conf_matrix_percentage, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix (Percentage)')
    plt.show()

def plot_roc_curve(y_true, y_pred_prob, class_names):
    y_true_bin = label_binarize(y_true, classes=[-1, 0, 1])

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i, class_label in enumerate(class_names):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_prob[:, i])
        roc_auc[i] = roc_auc_score(y_true_bin[:, i], y_pred_prob[:, i])

    plt.figure(figsize=(4, 3))
    for i, class_label in enumerate(class_names):
        plt.plot(fpr[i], tpr[i], label=f'Class {class_names[class_label]} (area = {roc_auc[i]:0.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()
    
def train_and_evaluate_model(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)

    # Predict discrete labels
    y_pred = model.predict(X_test)

    # Generate classification report
    report_dict = classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    print(report_df)

    # Compute custom accuracy
    custom_acc = custom_accuracy(y_test, y_pred)
    print(f'Custom Accuracy: {custom_acc:.2f}')

    # Plot confusion matrix
    class_names = {-1: 'Negative', 0: 'Neutral', 1: 'Positive'}
    plot_confusion_matrix(y_test, y_pred, list(class_names.values()))

    # Compute probabilities for ROC curve
    y_pred_prob = model.predict_proba(X_test)

    # Plot ROC curve
    plot_roc_curve(y_test, y_pred_prob, class_names)

In [ ]:
model = LogisticRegression(max_iter=300, solver='lbfgs', multi_class='multinomial')
train_and_evaluate_model(X_train_TI, y_train, X_test_TI, y_test, model)

In [ ]:
model = LogisticRegression(max_iter=300, solver='lbfgs', multi_class='multinomial')
train_and_evaluate_model(X_train_SV, y_train, X_test_SV, y_test, model)

### ML model: SLTM

In [ ]:
def create_and_train_lstm_model(X_train, y_train, X_test, y_test, epochs):
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(y_train.shape[1], activation='softmax'))  

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=1)
    
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)

    # Generate classification report
    report_dict = classification_report(y_test_labels, y_pred, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    print(f'Classification Report for {epochs} Epochs:\n', report_df)

    # Custom accuracy
    custom_acc = custom_accuracy(y_test_labels, y_pred)
    print(f'Custom Accuracy for {epochs} Epochs: {custom_acc:.2f}')

    # Plot confusion matrix
    class_names = [-1, 0, 1]
    plot_confusion_matrix(y_test_labels, y_pred, class_names)

    # Plot ROC curve
    plot_roc_curve(y_test_labels, y_pred_prob, class_names)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)  
y_test = lb.transform(y_test) 

# Train and evaluate the model with different numbers of epochs
for epochs in [50]:
    print(f'\nTraining LSTM Model with {epochs} Epochs')
    create_and_train_lstm_model(X_train_TI, y_train, X_test_TI, y_test, epochs)